This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/demand_curve).

## Demand Curve Pipeline Tutorial

This worksheet demonstrates a Wallaroo pipeline with data preprocessing, a model, and data postprocessing.

The model is a "demand curve" that predicts the expected number of units of a product that will be sold to a customer as a function of unit price and facts about the customer. Such models can be used for price optimization or sales volume forecasting.  This is purely a "toy" demonstration, but is useful for detailing the process of working with models and pipelines.

Data preprocessing is required to create the features used by the model. Simple postprocessing prevents nonsensical estimates (e.g. negative units sold).

## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed:
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.

In [104]:
import json
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas
import numpy
import conversion
from wallaroo.object import EntityNotFoundError
import pyarrow as pa

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [105]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

Now that the Wallaroo client has been initialized, we can create the workspace and call it `demandcurveworkspace`, then set it as our current workspace.  We'll also create our pipeline so it's ready when we add our models to it.

We'll set some variables and methods to create our workspace, pipelines and models.  Note that as of the July 2022 release of Wallaroo, workspace names must be unique.  Pipelines with the same name will be created as a new version when built.

In [106]:
workspace_name = 'demandcurveworkspace'
pipeline_name = 'demandcurvepipeline'
model_name = 'demandcurvemodel'
model_file_name = './demand_curve_v1.onnx'

In [107]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [108]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

demandcurve_pipeline = get_pipeline(pipeline_name)
demandcurve_pipeline

name,demandcurvepipeline
created,2024-02-13 17:42:28.947252+00:00
last_updated,2024-02-13 19:00:43.226475+00:00
deployed,False
arch,None
accel,None
tags,
versions,"f82b1315-ad7c-486a-9984-997596911cb3, cff86420-76e7-4204-bebd-bca35c343bc1, 59e8b52f-0f55-45cd-b348-31926cfffcf1, 55334083-061d-451d-a517-c02af05b71a8, b5ac75a7-2e07-4ed3-b81d-457a094eaa41, 10e7ae2d-369d-4bb3-84c8-3ccdcb85dd55, fe408a5c-0d73-4340-be88-66aa4e4e070a, 78126450-25b9-4a5c-a07e-7a183c144ae1, a0ed67d5-32c0-4d09-83eb-56690186931c, d14c9f6f-f29f-42bd-b94a-b5cd24eb6fde, e79b3f83-436e-4b75-b24b-d7eea3e1ea0a, 3380490c-3862-49ad-9d88-6b8bc2c3b1fb, 4ec8cd65-0690-44a9-8a50-babe78fb0f93, 1d7baa00-4d07-44d7-a7a6-4ec2f1a20580, d7e7e52a-5a25-4fd6-96c5-c223b167ea18"
steps,curve-preprocess
published,False


With our workspace established, we'll upload two models:

* `preprocess.py`: A Python step that converts the incoming DataFrame into tensor values.
* `demand_curve_v1.onnx`: Our demand_curve model.  We'll store the upload configuration into `demand_curve_model`.

Note that the order we upload our models isn't important - we'll be establishing the actual process of moving data from one model to the next when we set up our pipeline.

In [109]:
demand_curve_model = (wl.upload_model(model_name, 
                                      model_file_name, 
                                      framework=wallaroo.framework.Framework.ONNX)
                                      .configure(tensor_fields=["tensor"])
                    )

preprocess_input_schema = pa.schema([
    pa.field('Date', pa.string()),
    pa.field('cust_known', pa.bool_()),
    pa.field('StockCode', pa.int32()),
    pa.field('UnitPrice', pa.float32()),
    pa.field('UnitsSold', pa.int32())
])

preprocess_input_output = pa.schema([
    pa.field('tensor', pa.list_(pa.float64()))
])

preprocess_step = (wl.upload_model('curve-preprocess', 
                        './preprocess.py', 
                        framework=wallaroo.framework.Framework.PYTHON)
                        .configure(
                            'python', 
                            input_schema=preprocess_input_schema, 
                            output_schema=preprocess_input_output
                        )
        )

With our models uploaded, we're going to create our own pipeline and give it three steps:

* First, we apply the data to our `demand_curve_model`.

In [110]:
# now make a pipeline
demandcurve_pipeline.undeploy()
demandcurve_pipeline.clear()
demandcurve_pipeline.add_model_step(preprocess_step)
demandcurve_pipeline.add_model_step(demand_curve_model)

name,demandcurvepipeline
created,2024-02-13 17:42:28.947252+00:00
last_updated,2024-02-13 19:00:43.226475+00:00
deployed,False
arch,None
accel,None
tags,
versions,"f82b1315-ad7c-486a-9984-997596911cb3, cff86420-76e7-4204-bebd-bca35c343bc1, 59e8b52f-0f55-45cd-b348-31926cfffcf1, 55334083-061d-451d-a517-c02af05b71a8, b5ac75a7-2e07-4ed3-b81d-457a094eaa41, 10e7ae2d-369d-4bb3-84c8-3ccdcb85dd55, fe408a5c-0d73-4340-be88-66aa4e4e070a, 78126450-25b9-4a5c-a07e-7a183c144ae1, a0ed67d5-32c0-4d09-83eb-56690186931c, d14c9f6f-f29f-42bd-b94a-b5cd24eb6fde, e79b3f83-436e-4b75-b24b-d7eea3e1ea0a, 3380490c-3862-49ad-9d88-6b8bc2c3b1fb, 4ec8cd65-0690-44a9-8a50-babe78fb0f93, 1d7baa00-4d07-44d7-a7a6-4ec2f1a20580, d7e7e52a-5a25-4fd6-96c5-c223b167ea18"
steps,curve-preprocess
published,False


And with that - let's deploy our model pipeline.  This usually takes about 45 seconds for the deployment to finish.

In [111]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("1Gi").build()
demandcurve_pipeline.deploy(deployment_config=deploy_config)

name,demandcurvepipeline
created,2024-02-13 17:42:28.947252+00:00
last_updated,2024-02-13 19:05:33.325520+00:00
deployed,True
arch,None
accel,None
tags,
versions,"ab0bb94b-6247-4fab-bab5-be45f60e44d0, f82b1315-ad7c-486a-9984-997596911cb3, cff86420-76e7-4204-bebd-bca35c343bc1, 59e8b52f-0f55-45cd-b348-31926cfffcf1, 55334083-061d-451d-a517-c02af05b71a8, b5ac75a7-2e07-4ed3-b81d-457a094eaa41, 10e7ae2d-369d-4bb3-84c8-3ccdcb85dd55, fe408a5c-0d73-4340-be88-66aa4e4e070a, 78126450-25b9-4a5c-a07e-7a183c144ae1, a0ed67d5-32c0-4d09-83eb-56690186931c, d14c9f6f-f29f-42bd-b94a-b5cd24eb6fde, e79b3f83-436e-4b75-b24b-d7eea3e1ea0a, 3380490c-3862-49ad-9d88-6b8bc2c3b1fb, 4ec8cd65-0690-44a9-8a50-babe78fb0f93, 1d7baa00-4d07-44d7-a7a6-4ec2f1a20580, d7e7e52a-5a25-4fd6-96c5-c223b167ea18"
steps,curve-preprocess
published,False


We can check the status of our pipeline to make sure everything was set up correctly:

In [112]:
demandcurve_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.100.1.151',
   'name': 'engine-66f647b6ff-j8l5p',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'demandcurvepipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'curve-preprocess',
      'version': '093d64b7-8280-4b5c-b43d-120115445252',
      'sha': '3b2b32fb6408ccee5a1886ef5cdb493692080ff6699f49de518b20d9a6f6a133',
      'status': 'Running'},
     {'name': 'demandcurvemodel',
      'version': '0eeec65f-1efc-4604-b2e5-f1c0f5b00a96',
      'sha': '2820b42c9e778ae259918315f25afc8685ecab9967bad0a3d241e6191b414a0d',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.100.0.87',
   'name': 'engine-lb-dcd9c8cd7-76mhp',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

Everything is ready.  Let's feed our pipeline some data.  We have some information prepared with the `daily_purchasses.csv` spreadsheet.  We'll start with just one row to make sure that everything is working correctly.

In [113]:
# read in some purchase data
purchases = pandas.read_csv('daily_purchases.csv')

# start with a one-row data frame for testing
subsamp_raw = purchases.iloc[0:1,: ]
subsamp_raw

,Date,cust_known,StockCode,UnitPrice,UnitsSold
0,2010-12-01,False,21928,4.21,1


In [114]:
result = demandcurve_pipeline.infer(subsamp_raw)
display(result)

,time,in.Date,in.StockCode,in.UnitPrice,in.UnitsSold,in.cust_known,out.variable,anomaly.count
0,2024-02-13 19:05:54.701,2010-12-01,21928,4.21,1,False,[6.680255142999893],0


We can see from the `out.variable` field that the demand curve has a predicted slope of 6.68 from our sample data.  We can isolate that by specifying just the data output below.

In [115]:
display(result.loc[0, ['out.variable']][0])

[6.680255142999893]

# Bulk Inference

The initial test went perfectly.  Now let's throw some more data into our pipeline.  We'll draw 10 random rows from our spreadsheet, perform an inference from that, and then display the results and the logs showing the pipeline's actions.

In [116]:
ix = numpy.random.choice(purchases.shape[0], size=10, replace=False)
converted = conversion.pandas_to_dict(purchases.iloc[ix,: ])
test_df = pd.DataFrame(converted['query'], columns=converted['colnames'])
display(test_df)

output = demandcurve_pipeline.infer(test_df)
display(output)

,Date,cust_known,StockCode,UnitPrice,UnitsSold
0,2011-10-31,True,20711,2.08,10
1,2011-08-26,True,23203,2.08,44
2,2011-05-06,False,21930,4.13,7
3,2011-05-05,False,22385,2.08,5
4,2011-05-22,True,22385,2.08,1
5,2011-08-18,True,20712,2.08,50
6,2011-11-16,False,21931,4.13,7
7,2010-12-07,False,85099F,4.21,1
8,2011-09-04,True,21929,2.08,12
9,2011-05-12,True,23201,1.95,10


,time,in.Date,in.StockCode,in.UnitPrice,in.UnitsSold,in.cust_known,out.variable,anomaly.count
0,2024-02-13 19:05:55.139,2011-10-31,20711,2.08,10,True,[33.125327529877765],0
1,2024-02-13 19:05:55.139,2011-08-26,23203,2.08,44,True,[33.125327529877765],0
2,2024-02-13 19:05:55.139,2011-05-06,21930,4.13,7,False,[6.7715457962084376],0
3,2024-02-13 19:05:55.139,2011-05-05,22385,2.08,5,False,[9.110871233285868],0
4,2024-02-13 19:05:55.139,2011-05-22,22385,2.08,1,True,[33.125327529877765],0
5,2024-02-13 19:05:55.139,2011-08-18,20712,2.08,50,True,[33.125327529877765],0
6,2024-02-13 19:05:55.139,2011-11-16,21931,4.13,7,False,[6.7715457962084376],0
7,2024-02-13 19:05:55.139,2010-12-07,85099F,4.21,1,False,[6.680255142999893],0
8,2024-02-13 19:05:55.139,2011-09-04,21929,2.08,12,True,[33.125327529877765],0
9,2024-02-13 19:05:55.139,2011-05-12,23201,1.95,10,True,[40.57067616108544],0


## Undeploy the Pipeline

Once we've finished with our demand curve demo, we'll undeploy the pipeline and give the resources back to our Kubernetes cluster.

In [118]:
demandcurve_pipeline.undeploy()

name,demandcurvepipeline
created,2024-02-13 17:42:28.947252+00:00
last_updated,2024-02-13 19:05:33.325520+00:00
deployed,False
arch,None
accel,None
tags,
versions,"ab0bb94b-6247-4fab-bab5-be45f60e44d0, f82b1315-ad7c-486a-9984-997596911cb3, cff86420-76e7-4204-bebd-bca35c343bc1, 59e8b52f-0f55-45cd-b348-31926cfffcf1, 55334083-061d-451d-a517-c02af05b71a8, b5ac75a7-2e07-4ed3-b81d-457a094eaa41, 10e7ae2d-369d-4bb3-84c8-3ccdcb85dd55, fe408a5c-0d73-4340-be88-66aa4e4e070a, 78126450-25b9-4a5c-a07e-7a183c144ae1, a0ed67d5-32c0-4d09-83eb-56690186931c, d14c9f6f-f29f-42bd-b94a-b5cd24eb6fde, e79b3f83-436e-4b75-b24b-d7eea3e1ea0a, 3380490c-3862-49ad-9d88-6b8bc2c3b1fb, 4ec8cd65-0690-44a9-8a50-babe78fb0f93, 1d7baa00-4d07-44d7-a7a6-4ec2f1a20580, d7e7e52a-5a25-4fd6-96c5-c223b167ea18"
steps,curve-preprocess
published,False


Thank you for being a part of this demonstration.  If you have additional questions, please feel free to contact us at Wallaroo.